In [33]:
import pandas as pd

# Erläuterungen: https://www.arcgis.com/home/item.html?id=dd4580c810204019a7b8eb3e0b329dd6

# > 250 MB, braucht seine Zeit, das zu laden
df = pd.read_csv("https://opendata.arcgis.com/api/v3/datasets/dd4580c810204019a7b8eb3e0b329dd6_0/downloads/data?format=csv&spatialRefId=4326")

bak = df

In [ ]:
df.head(10)

In [ ]:
# Wegen korrigierter Daten führt einfaches Addieren zu leicht inkorrekten Daten...

df[ ["AnzahlFall", "AnzahlTodesfall"] ].sum()

In [ ]:
# NeuerFall: 
#   0: Fall ist in der Publikation für den aktuellen Tag und in der für den Vortag enthalten
#   1: Fall ist nur in der aktuellen Publikation enthalten
#  -1: Fall ist nur in der Publikation des Vortags enthalten

# damit ergibt sich: Anzahl Fälle der aktuellen Publikation als Summe(AnzahlFall), 
# wenn NeuerFall in (0,1); Delta zum Vortag als Summe(AnzahlFall) wenn NeuerFall in (-1,1)

# -> die neuen minus die korrigierten Fälle gibt die Erhöhung des heutigen Tages

alte = df[ (df["NeuerFall"] == 0) ]["AnzahlFall"].sum()
#print(alte)

neue = df[ (df["NeuerFall"] == 1) ]["AnzahlFall"].sum()
#print(neue)

korrektur = df[ (df["NeuerFall"] == -1) ]["AnzahlFall"].sum()
#print(korrektur)

print("Differenz zum Vortag: " + str(neue + korrektur))
print("Gesamt: " + str(neue + alte))

In [ ]:
# Und jetzt pro Bundesland

alte = df[ (df["NeuerFall"] == 0) ].groupby("Bundesland")["Bundesland", "AnzahlFall"].sum()
print(alte)

neue = df[ (df["NeuerFall"] == 1) ].groupby("Bundesland")["Bundesland", "AnzahlFall"].sum()
print(neue)

korrektur = df[ (df["NeuerFall"] == -1) ].groupby("Bundesland")["Bundesland", "AnzahlFall"].sum()
print(korrektur)

#print(neue + alte)

In [36]:
# wieder geht es nicht so einfach... wenn Bundesländer keine Zahlen oder keine Korrektur melden, gibt es NaN...

neue.set_axis(["AnzahlFallHeute"], axis=1, inplace=True)
korrektur.set_axis(["AnzahlKorrekturHeute"], axis=1, inplace=True)

heute = pd.concat([neue, korrektur], axis=1, sort=False).fillna(0).astype(int)

gesamt = alte.copy()

gesamt["DiffZuVortag"] = heute["AnzahlFallHeute"] + heute["AnzahlKorrekturHeute"]
gesamt["AnzahlFallHeute"] = heute["AnzahlFallHeute"]

gesamt["AnzahlFall"] = gesamt[["AnzahlFall", "AnzahlFallHeute"]].sum(axis=1)
gesamt.drop(gesamt.columns[2], axis=1, inplace=True)

gesamt = gesamt.fillna(0).astype(int)
print(gesamt)

                        AnzahlFall  DiffZuVortag
Bundesland                                      
Baden-Württemberg           487776           266
Bayern                      634746           357
Berlin                      177305            76
Brandenburg                 107820            21
Bremen                       26883            15
Hamburg                      75817            68
Hessen                      283959            89
Mecklenburg-Vorpommern       43610            10
Niedersachsen               256233            71
Nordrhein-Westfalen         798651           538
Rheinland-Pfalz             151493           160
Saarland                     40429            16
Sachsen                     281905            40
Sachsen-Anhalt               97760            37
Schleswig-Holstein           62622            80
Thüringen                   126542            67


In [ ]:
# Summe der letzte 7 Tage

from datetime import timedelta

df["Meldedatum"]= pd.to_datetime(df["Meldedatum"]).dt.tz_localize(None)
df["Datenstand"]= pd.to_datetime(df["Datenstand"], format="%d.%m.%Y, %H:%M Uhr")
df["Refdatum"]= pd.to_datetime(df["Refdatum"]).dt.tz_localize(None)
print(df["Meldedatum"].max())

wochenfrist = df["Meldedatum"].max() - timedelta(days=7)

gefiltert = df[ ((df["NeuerFall"] == 0) | (df["NeuerFall"] == 1)) & (df["Meldedatum"] > wochenfrist)]

gesamt["Summe7Tage"] = gefiltert.groupby("Bundesland")["Bundesland", "AnzahlFall"].sum()

gesamt

In [ ]:
# Inzidenz benötigt die Einwohnerzahlen der Bundesländer -> in einem zukünftigen Video ;-)

In [ ]:
# Todesfälle pro Bundesland

tfs = df[ (df["NeuerTodesfall"] == 0) | (df["NeuerTodesfall"] == 1) ]

gesamt["AnzahlTf"] = df.groupby("Bundesland")["Bundesland", "AnzahlTodesfall"].sum()

gesamt

In [ ]:
# Summen für ganz Deutschland

summen = gesamt.sum()

brd = pd.DataFrame({"":summen.index, "Bundesrepublik":summen.values}).transpose()

brd.columns = brd.iloc[0]
brd = brd.drop(brd.index[0])

brd = pd.concat([gesamt, brd])

brd

In [ ]:
# Grafische Darstellung der Fallzahlen

import matplotlib.pylab as plt
#import matplotlib.dates as md

gefiltert = df[(df["NeuerFall"] == 1) | (df["NeuerFall"] == 0)]

summe = gefiltert.groupby("Meldedatum")["AnzahlFall"].sum()
#schnitt = summe.rolling(7).mean()

ax = summe.plot(figsize=(20,15))
schnitt.plot(ax=ax)

#ax.legend(["Fallzahlen", "7TageSchnitt"])

#ax.set_axisbelow(True)
#ax.grid()
#ax.xaxis.set_major_locator(md.MonthLocator(interval=1))
#ax.xaxis.set_major_formatter(md.DateFormatter("%d-%m-%Y"))

plt.show()